In [ ]:
!pip install --upgrade --no-cache-dir gdown

  Attempting uninstall: gdown
    Found existing installation: gdown 4.7.3
    Uninstalling gdown-4.7.3:
      Successfully uninstalled gdown-4.7.3


In [ ]:
import gdown
url = "https://drive.google.com/drive/folders/1L_6hJY0ye2VbSFhq2B1Rt34BbWj89WEQ"



gdown.download_folder(url, quiet=True, use_cookies=False)


['/content/OGLE3_clean/ogle3-acep.zip',
 '/content/OGLE3_clean/ogle3-acv.zip',
 '/content/OGLE3_clean/ogle3-cep.zip',
 '/content/OGLE3_clean/ogle3-dn.zip',
 '/content/OGLE3_clean/ogle3-dpv.zip',
 '/content/OGLE3_clean/ogle3-dsct.zip',
 '/content/OGLE3_clean/ogle3-ecl.zip',
 '/content/OGLE3_clean/ogle3-lpv.zip',
 '/content/OGLE3_clean/ogle3-rcb.zip',
 '/content/OGLE3_clean/ogle3-rrlyr.zip',
 '/content/OGLE3_clean/ogle3-t2cep.zip',
 '/content/OGLE3_clean/ogle3-wd.zip',
 '/content/OGLE3_clean/ogle3-yso.zip']

In [10]:
import os
import zipfile
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import random
import shutil

In [ ]:
seed = 42
np.random.seed(seed)
random.seed(seed)

In [ ]:
def find_zip_files(directory):
    """
    Find zip files within a directory.

    Args:
    - directory (str): The directory to search for zip files.

    Returns:
    - list: A list of paths to zip files found in the directory.
    """
    zip_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".zip"):
                zip_files.append(os.path.join(root, file))
    return zip_files

In [ ]:
root_folder = '/content/OGLE3_clean'

In [ ]:
zip_files = find_zip_files(root_folder)
print(zip_files)

['/content/OGLE3_clean/ogle3-acv.zip', '/content/OGLE3_clean/ogle3-t2cep.zip', '/content/OGLE3_clean/ogle3-yso.zip', '/content/OGLE3_clean/ogle3-rcb.zip', '/content/OGLE3_clean/ogle3-ecl.zip', '/content/OGLE3_clean/ogle3-acep.zip', '/content/OGLE3_clean/ogle3-rrlyr.zip', '/content/OGLE3_clean/ogle3-wd.zip', '/content/OGLE3_clean/ogle3-dn.zip', '/content/OGLE3_clean/ogle3-dsct.zip', '/content/OGLE3_clean/ogle3-dpv.zip', '/content/OGLE3_clean/ogle3-cep.zip', '/content/OGLE3_clean/ogle3-lpv.zip']


In [ ]:
for zip_file in zip_files:
    #!unzip -q "$zip_file" -d "{os.path.splitext(zip_file)[0]}"
    !unzip -q "$zip_file" -d "{os.path.dirname(os.path.splitext(zip_file)[0])}"
    print("extraction done for "+ zip_file)

extraction done for /content/OGLE3_clean/ogle3-acv.zip
extraction done for /content/OGLE3_clean/ogle3-t2cep.zip
extraction done for /content/OGLE3_clean/ogle3-yso.zip
extraction done for /content/OGLE3_clean/ogle3-rcb.zip
extraction done for /content/OGLE3_clean/ogle3-ecl.zip
extraction done for /content/OGLE3_clean/ogle3-acep.zip
extraction done for /content/OGLE3_clean/ogle3-rrlyr.zip
extraction done for /content/OGLE3_clean/ogle3-wd.zip
extraction done for /content/OGLE3_clean/ogle3-dn.zip
extraction done for /content/OGLE3_clean/ogle3-dsct.zip
extraction done for /content/OGLE3_clean/ogle3-dpv.zip
extraction done for /content/OGLE3_clean/ogle3-cep.zip
extraction done for /content/OGLE3_clean/ogle3-lpv.zip


In [ ]:
for zip_file in zip_files:
  !rm -r "$zip_file"
  print("delete done for "+ zip_file)

delete done for /content/OGLE3_clean/ogle3-acv.zip
delete done for /content/OGLE3_clean/ogle3-t2cep.zip
delete done for /content/OGLE3_clean/ogle3-yso.zip
delete done for /content/OGLE3_clean/ogle3-rcb.zip
delete done for /content/OGLE3_clean/ogle3-ecl.zip
delete done for /content/OGLE3_clean/ogle3-acep.zip
delete done for /content/OGLE3_clean/ogle3-rrlyr.zip
delete done for /content/OGLE3_clean/ogle3-wd.zip
delete done for /content/OGLE3_clean/ogle3-dn.zip
delete done for /content/OGLE3_clean/ogle3-dsct.zip
delete done for /content/OGLE3_clean/ogle3-dpv.zip
delete done for /content/OGLE3_clean/ogle3-cep.zip
delete done for /content/OGLE3_clean/ogle3-lpv.zip


In [ ]:
!ls /content/OGLE3_clean

acep  acv  cep	dn  dpv  dsct  ecl  lpv  rcb  rrlyr  t2cep  wd	yso


##augmentation

In [ ]:
def print_folder_counts(root_folder):
    for class_folder in sorted(os.listdir(root_folder)):  # Sorted for consistent order
        class_folder_path = os.path.join(root_folder, class_folder)
        if os.path.isdir(class_folder_path):  # Ensure it's a folder
            num_files = len([f for f in os.listdir(class_folder_path) if f.endswith('.dat')])
            print(f"{class_folder}: {num_files} files")

In [ ]:
# Print counts before augmentation
print("Before augmentation:")
print_folder_counts(root_folder)

Before augmentation:
acep: 83 files
acv: 5 files
cep: 8026 files
dn: 40 files
dpv: 137 files
dsct: 2859 files
ecl: 43859 files
lpv: 343816 files
rcb: 23 files
rrlyr: 44262 files
t2cep: 603 files
wd: 1 files
yso: 3 files


In [ ]:
def augment_light_curves(data_path, output_path):
    # Read the original data
    data = pd.read_csv(data_path, sep="\s+", header=None)
    data.columns = ['Time', 'Magnitude', 'Err']

    # Augment the data and save as supplementary .dat files
    step = random.randint(1, 2)
    burning = random.randint(0, 5)

    augmented_data = np.roll(data.values, step, axis=0)[burning:]
    np.savetxt(output_path, augmented_data, delimiter='\t', fmt='%f')


In [ ]:
def augment_class_folders(root_folder, n_max=8000, max_replications=5):
    for class_folder in tqdm(os.listdir(root_folder)) :#, desc= f"working on {class_folder}", end = ""):
        class_folder_path = os.path.join(root_folder, class_folder)

        dat_files = [f for f in os.listdir(class_folder_path) if f.endswith('.dat')]
        num_files = len(dat_files)
        num_augmented_needed = max(0, n_max - num_files)

        if num_augmented_needed > 0:
            replication_count = {file: 0 for file in dat_files}
            while num_augmented_needed > 0 and any(replication_count[file] < max_replications for file in dat_files):
                for dat_file in random.sample(dat_files, len(dat_files)):
                    if num_augmented_needed <= 0 or replication_count[dat_file] >= max_replications:
                        continue

                    data_path = os.path.join(class_folder_path, dat_file)
                    output_file = os.path.join(class_folder_path, f"{os.path.splitext(dat_file)[0]}_augmented_{replication_count[dat_file]}.dat")
                    augment_light_curves(data_path, output_file)

                    replication_count[dat_file] += 1
                    num_augmented_needed -= 1
        #print()
    print(f"Finished augmenting folder" )#end_time.strftime('%Y-%m-%d %H:%M:%S')}")



In [ ]:
augment_class_folders(root_folder)


100%|██████████| 13/13 [00:52<00:00,  4.07s/it]

Finished augmenting folder


In [ ]:
# Print counts before augmentation
print("After augmentation:")
print_folder_counts(root_folder)

After augmentation:
acep: 498 files
acv: 30 files
cep: 8026 files
dn: 240 files
dpv: 822 files
dsct: 8000 files
ecl: 43859 files
lpv: 343816 files
rcb: 138 files
rrlyr: 44262 files
t2cep: 3618 files
wd: 6 files
yso: 18 files


##preprocessing

In [ ]:
#!rm -r /content/OGLE3

In [ ]:
# Function to preprocess light curves
def preprocess_light_curves(data_path):
    # Read data from fixed-width file
    #"""
    data = pd.read_csv(data_path, sep = "\s+", header=None).reset_index(drop=True)
    data.columns = ['Time', 'Magnitude', 'Err']
    data = data.astype('float64')
    data = data[data.Time > 0]
    data.sort_values(by=['Time'])
    data.drop_duplicates(subset='Time', keep='first')



    # Compute differences between consecutive observations for time and magnitude
    # Note: diff() leaves the first element as NaN, which we replace with zeros
    data['Time_diff'] = data['Time'].diff().fillna(0)
    data['Magnitude_diff'] = data['Magnitude'].diff().fillna(0)

    # Prepare for padding if necessary, targeting a minimum of 500 observations
    min_observations = 500
    padding_needed = max(0, min_observations - len(data))

    # If padding is needed, extend the dataframe with rows of zeros
    if padding_needed > 0:
        padding_df = pd.DataFrame({
            'Time': [0] * padding_needed,
            'Magnitude': [0] * padding_needed,
            'Err': [0] * padding_needed,
            'Time_diff': [0] * padding_needed,
            'Magnitude_diff': [0] * padding_needed
        })
        data = pd.concat([data, padding_df], ignore_index=True)

    # After padding, if the dataset is larger than the target size, truncate it to the target size
    data = data.head(min_observations)

    # Since we're only interested in Time_diff and Magnitude_diff for the network input,
    # we reshape our data accordingly to a shape of 1 × 2 × N (1 light curve, 2 features, N observations)
    matrix_representation = np.stack([data['Time_diff'].values, data['Magnitude_diff'].values], axis=0).reshape(1, 2, -1)

    return matrix_representation

In [ ]:
def preprocess_class_folders(root_folder, output_folder, preprocess_func):
    """
    For each .dat file in the class folders, apply a preprocessing function and save the output as a CSV file.
    Each light curve is saved to its own CSV file in a specified output directory.

    Args:
    - root_folder (str): Path to the directory containing class folders with .dat files.
    - output_folder (str): Path to the directory where preprocessed folders and files will be stored.
    - preprocess_func (function): The preprocessing function to be applied to each .dat file.
    """
    # Iterate over each class folder in the root directory
    for class_folder in os.listdir(root_folder):
        class_folder_path = os.path.join(root_folder, class_folder)

        # Only process directories
        if os.path.isdir(class_folder_path):
            # Create a new directory for preprocessed files within the specified output folder
            preprocessed_folder_path = os.path.join(output_folder, f"{class_folder}_preprocessed")
            if not os.path.exists(preprocessed_folder_path):
                os.makedirs(preprocessed_folder_path)

            all_dat_files = [f for f in os.listdir(class_folder_path) if f.endswith('.dat')]

            # If more than 8000 .dat files exist, randomly select 8000 without repetition
            if len(all_dat_files) > 8000:
                selected_files = random.sample(all_dat_files, 8000)
            else:
                selected_files = all_dat_files

            # Process each selected .dat file within the class folder
            for dat_file in tqdm(selected_files, desc=f"Processing {class_folder}"):
                dat_file_path = os.path.join(class_folder_path, dat_file)
                preprocessed_data = preprocess_func(dat_file_path)

                # Flatten the preprocessed data for saving
                preprocessed_data_flat = preprocessed_data.reshape(2, -1).T  # Reshape to Nx2 for saving

                # Define the new filename and path for the preprocessed file
                preprocessed_file_name = f"{os.path.splitext(dat_file)[0]}_preprocessed.csv"
                preprocessed_file_path = os.path.join(preprocessed_folder_path, preprocessed_file_name)

                # Save the preprocessed data to the new file
                np.savetxt(preprocessed_file_path, preprocessed_data_flat, delimiter=',', header='time_diff,mag_diff', comments='')

            print(f"Completed processing for class: {class_folder}.")
    print("Preprocessing complete.")


In [ ]:
output_folder = '/content/drive/MyDrive/ADV_ML/OGLE3_clean_preprocessed'

In [ ]:
preprocess_class_folders(root_folder, output_folder, preprocess_light_curves)

Processing dpv: 100%|██████████| 822/822 [00:20<00:00, 40.86it/s]


Completed processing for class: dpv.


Processing acv: 100%|██████████| 30/30 [00:00<00:00, 43.81it/s]


Completed processing for class: acv.


Processing lpv: 100%|██████████| 8000/8000 [03:29<00:00, 38.19it/s]


...

Completed processing for class: rcb.


Processing rrlyr: 100%|██████████| 8000/8000 [03:39<00:00, 36.44it/s]


Completed processing for class: rrlyr.


Processing t2cep: 100%|██████████| 3618/3618 [01:52<00:00, 32.15it/s]

Completed processing for class: t2cep.
Preprocessing complete.
